In [51]:
import numpy as np

#load in data
Xp = np.array([[4, 2, 2, 3, 4, 6, 3, 8], [1, 4, 3, 6, 4, 2, 2, 3], [0, 1, 1, 0, -1, 0, 1, 0]])
Xn = np.array([[9, 6, 9, 8, 10], [10, 8, 5, 7, 8], [1, 0, 0, 1, -1]])
npos = np.size(Xp,1)
nneg = np.size(Xn,1)

#compute mean of data
mup = meanp(Xp)
print(f'mean of positive data : \n{mup}\n')
mun = meann(Xn)
print(f'mean of negative data : \n{mun}\n')

#compute covariance matrices
Cp = covariance(Xp, mup)
print(f'covariance matrix for Xp: \n{Cp}\n')
Cn = covariance(Xn, mun)
print(f'covariance matrix for Xn: \n{Cn}\n')

#compute between class scattering matrix
Sb = BCSM(npos, nneg, mup, mun)
print(f'between class scattering matrix: \n{Sb}\n')

#compute within class scattering matrix
Sw = WCSM(npos, nneg, Cp, Cn)
print(f'within class scattering matrix: \n{Sw}\n')

#compute LDA projection
v = solveLDA(Sb, Sw)
print(f'projection direction: \n{v}\n')

#compute optimal LDA projection direction of two data matrices
w = mybLDA_train(Xp, Xn)
print(f'projection direction: \n{w}\n')

#load in data to be classified
X = np.array([[1.3, 2.4, 6.7, 2.2, 3.4, 3.2], [8.1, 7.6, 2.1, 1.1, 0.5, 7.4], [-1, 2, 3, -2, 0, 2]])

#classify data
r = mybLDA_classify(X,v)
print(f'classification of X: \n{r}\n')

mean of positive data : 
[[4.   ]
 [3.125]
 [0.25 ]]

mean of negative data : 
[[-8.4]
 [-7.6]
 [-0.2]]

covariance matrix for Xp: 
[[ 3.75     -0.75     -0.625   ]
 [-0.75      2.109375 -0.15625 ]
 [-0.625    -0.15625   0.4375  ]]

covariance matrix for Xn: 
[[284.08 255.32   6.44]
 [255.32 233.68   6.36]
 [  6.44   6.36   0.72]]

between class scattering matrix: 
[[36.39289941 31.47692308  1.32071006]
 [31.47692308 27.225       1.14230769]
 [ 1.32071006  1.14230769  0.04792899]]

within class scattering matrix: 
[[111.56923077  97.73846154   2.09230769]
 [ 97.73846154  91.175        2.35      ]
 [  2.09230769   2.35         0.54615385]]

projection direction: 
[ 0.3084552  -0.13425564  0.94171695]

projection direction: 
[ 0.3084552  -0.13425564  0.94171695]

classification of X: 
[-1.  1.  1. -1.  1.  1.]



In [3]:
def meanp(X):
    n = np.size(X,1)
    ones = np.ones((n,1))
    return (1/n)*(X.dot(ones))

In [6]:
def meann(X):
    n = np.size(X,1)
    ones = -1*np.ones((n,1))
    return (1/n)*(X.dot(ones))

In [11]:
def covariance(X, mu):
    n = np.size(X,1)
    return (1/n)*(X-mu).dot(np.transpose(X-mu))

In [19]:
def BCSM(npos, nneg, mup, mun):
    n = npos + nneg
    return ((npos*nneg)/n**2)*(mup-mun).dot(np.transpose(mup-mun))

In [22]:
def WCSM(npos, nneg, Cp, Cn):
    n = npos + nneg
    return (npos/n)*Cp+(nneg/n)*Cn

In [32]:
def solveLDA(Sb, Sw):
    Swi = np.linalg.inv(Sw)
    w, v = np.linalg.eig(Swi.dot(Sb))
    return v[:,0]

In [37]:
def mybLDA_train(Xp, Xn):
    npos = np.size(Xp,1)
    nneg = np.size(Xn,1)
    mup = meanp(Xp)
    mun = meann(Xn)
    Cp = covariance(Xp, mup)
    Cn = covariance(Xn, mun)
    Sb = BCSM(npos, nneg, mup, mun)
    Sw = WCSM(npos, nneg, Cp, Cn)
    return solveLDA(Sb, Sw)

In [50]:
def mybLDA_classify(X,v):
    n = np.size(X,1)
    r = np.zeros(n)
    for i in range(0,n):
        if (X[:,i].dot(v)) > 0:
            r[i] = 1
        else:
            r[i] = -1
    return r